In [7]:
%matplotlib widget
''' 系统库 '''
import datetime
import time
from random import randint 
from importlib import reload 
# from numba import jit
# @jit(nopython=True)

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

''' 业务库 '''
import mplfinance as mpf
import tushare as ts
ts.set_token('11137efdbeac800606d677871a55b3fd5aef79890c59867a8f34d03e')
pro = ts.pro_api()

''' 自建库 '''
from qflib import basic,syncdata
reload(basic)
reload(syncdata)
engine = basic.engine()


trade_date = datetime.datetime.now().strftime('%Y%m%d')
print('date : ', trade_date)
run_static, run_dynamic = False, True
# run_static, run_dynamic = True, True
# run_static, run_dynamic = True, False



date :  20230714


In [8]:
# 分钟数据，freq/数据频度 ：1MIN表示1分钟（1/5/15/30/60分钟） D日线 ，默认D， W，M;  *** 您每天最多访问该接口2次
# df = ts.pro_bar(ts_code='601318.SH', adj='qfq', freq='1min', start_date='20220401', end_date='20220430')
# print(df.head(3))
# df.to_csv('../data/minite_601318.csv')

# print( '更新基本信息' )
# df = pro.stock_basic(exchange='', list_status='L')
# syncdata.sync_basic(engine, df, 'stock_basic', trade_date)

print( "更新深股通清单")
df = pro.hs_const(hs_type='SZ')
syncdata.sync_basic(engine, df, 'sl_shengutong', trade_date)

# df = ts.pro_bar(ts_code='601318.SH', adj='qfq', start_date='20200101', end_date='20231231')
# df
# basic.write_data(engine,df,'sl_shengutong', False)

更新深股通清单
sl_shengutong <sqlalchemy.engine.cursor.LegacyCursorResult object at 0x000001E20F3CDE70>


In [9]:
%%time
"""
静态数据更新
"""

if run_static:

    print( '更新基本信息' )
    df = pro.stock_basic(exchange='', list_status='L')
    syncdata.sync_basic(engine, df, 'stock_basic', trade_date)

    print( "更新公司信息")
    df = pro.stock_company(exchange='', ts_code='')  
    syncdata.sync_basic(engine, df, 'stock_company', trade_date)
    
    print( "获得股票关键市场信息； 5000积分以上，否则每分钟最多2次, 每天20次；")
    df = pro.bak_basic(trade_date=trade_date)
    syncdata.sync_basic(engine, df, 'bak_basic', trade_date)

    print( "更新深股通清单")
    df = pro.hs_const(hs_type='SZ')
    syncdata.sync_basic(engine, df, 'sl_shengutong', trade_date)

    print("更新沪股通清单")
    df = pro.hs_const(hs_type='SH')
    syncdata.sync_basic(engine, df, 'sl_hugutong', trade_date)

    print("开始改名的股票")
    df = pro.namechange(ts_code='', start_date='2019-01-01', fields='ts_code,name,start_date,end_date,change_reason')
    syncdata.sync_basic(engine, df, 'namechange', trade_date)

    print("交易日历")
    df = pro.trade_cal(exchange='SSE', start_date='20190101', end_date='20221231')
    syncdata.sync_basic(engine, df, 'calendar', trade_date)
    
    print("股票概念清单")   # 879+个
    df = pro.concept(src='ts') 
    syncdata.sync_basic(engine, df, 'concept', trade_date)

    

CPU times: total: 0 ns
Wall time: 0 ns


In [10]:
%%time
"""
动态数据更新：每日成交、每日MoneyFlow
"""

days = 365*5

reload(basic)
reload(syncdata)

if run_dynamic:
    # 每日成交数据 -- OK
    syncdata.sync_tran_daily(engine, days)
    # 每日资金流向  --  暂停
    # syncdata.sync_moneyflow(engine, days)

    

20180716  - existed
20180717  - existed
20180718  - existed
20180719  - existed
20180720  - existed
20180721  - read:  (0, 11)  - none 
20180722  - read:  (0, 11)  - none 
20180723  - existed
20180724  - existed
20180725  - existed
20180726  - existed
20180727  - existed
20180728  - read:  (0, 11)  - none 
20180729  - read:  (0, 11)  - none 
20180730  - existed
20180731  - existed
20180801  - existed
20180802  - existed
20180803  - existed
20180804  - read:  (0, 11)  - none 
20180805  - read:  (0, 11)  - none 
20180806  - existed
20180807  - existed
20180808  - existed
20180809  - existed
20180810  - existed
20180811  - read:  (0, 11)  - none 
20180812  - read:  (0, 11)  - none 
20180813  - existed
20180814  - existed
20180815  - existed
20180816  - existed
20180817  - existed
20180818  - read:  (0, 11)  - none 
20180819  - read:  (0, 11)  - none 
20180820  - existed
20180821  - existed
20180822  - existed
20180823  - existed
20180824  - existed
20180825  - read:  (0, 11)  - none 
2018

In [11]:

# import cStringIO
 
# output = cStringIO.StringIO()
# # ignore the index
# df_a.to_csv(output, sep='\t',index = False, header = False)
# output.getvalue()
# # jump to start of stream
# output.seek(0)
 
# connection = engine.raw_connection() #engine 是 from sqlalchemy import create_engine
# cursor = connection.cursor()
# # null value become ''
# cursor.copy_from(output,table_name,null='')
# connection.commit()


In [12]:
'''管理层薪酬和持股; 需要高级权限 '''
# df = pro.stk_rewards(ts_code='601318.SH')

'''--- 新股上市清单， 120积分 '''
# df = pro.new_share(start_date='20220101', end_date='20221231')
# df.tail(10)

''' 概念股清单 
    58,TS58,锂电隔膜,ts
    59,TS59,锂电设备,ts
    60,TS60,锂电原料,ts
    61,TS61,锂电制造,ts
'''
# df = pro.concept_detail(id='TS60', fields='ts_code,name') 
# df.to_csv('~/win-desk/lidianyuanliao.txt', index=False, header=False)


' 概念股清单 \n    58,TS58,锂电隔膜,ts\n    59,TS59,锂电设备,ts\n    60,TS60,锂电原料,ts\n    61,TS61,锂电制造,ts\n'